In [12]:
import pickle

# Function to load and inspect pickle files
def load_and_inspect(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data

# Load and inspect the data files
train_infor_his = load_and_inspect('./data/train/train_infor_his')
val_id_history = load_and_inspect('./data/val/val_id_history')
val_interaction = load_and_inspect('./data/val/val_interaction')
train_interaction = load_and_inspect('./data/train/train_infor_his')
news_information = load_and_inspect('./data/news_infor')
test_information_history = load_and_inspect('./data/test/test_infor_his')
test_news_information = load_and_inspect('./data/test/test_news_infor')
test_interaction = load_and_inspect('./data/test/test_interaction')

print("news_information:", list(news_information.items())[:10])
print("train_infor_his:", list(train_infor_his.items())[:5])
print("train_interaction:", list(train_interaction.items())[:5])
print("val_id_history:", list(val_id_history.items())[:10])
print("val_interaction:", list(val_interaction)[:10])
print("test_information_history:", list(test_information_history.items())[:5])
print("test_news_information:", list(test_news_information.items())[:5])
print("test_interaction:", list(test_interaction)[:5])



news_information: [('N55528', 'lifestyleroyals The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By'), ('N19639', 'weightloss 50 Worst Habits For Belly Fat'), ('N61837', "newsworld The Cost of Trump's Aid Freeze in the Trenches of Ukraine's War"), ('N53526', "voices I Was An NBA Wife. Here's How It Affected My Mental Health."), ('N38324', 'medical How to Get Rid of Skin Tags, According to a Dermatologist'), ('N2073', 'football_nfl Should NFL be able to fine players for criticizing officiating?'), ('N49186', "weathertopstories It's been Orlando's hottest October ever so far, but cooler temperatures on the way"), ('N59295', 'newsworld Chile: Three die in supermarket fire amid protests'), ('N24510', 'gaming Best PS5 games: top PlayStation 5 titles to look forward to'), ('N39237', 'newsscienceandtechnology How to report weather-related closings, delays')]
train_infor_his: [('2', ["newscrime 'Habitual stowaway' arrested again at O'Hare, Chicago police say", 'newsus Search 

In [15]:
results = load_and_inspect('results.pkl')
print("results:", results)

results: [(['U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U80234', 'U60458', 'U60458', 'U60458', 'U60458', 'U60458', 'U60458', 'U60458', 'U60458', 'U60458', 'U60458', 'U60458', 'U60458', 'U60458', 'U60458', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190', 'U44190'

In [9]:
len(results[0])

6

In [16]:
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import ndcg_score

# Assuming results[0] has the following structure: user_id, impress_id, item_id, target_text, prob_yes, generated_sents
user_ids, impress_ids, item_ids, target_texts, prob_yes, generated_sents = results[0]

# Group data by user for metrics calculation
user_results = {}
for user, item, prob, response in zip(user_ids, item_ids, prob_yes, target_texts):
    if user not in user_results:
        user_results[user] = []
    user_results[user].append((item, prob, response))

auc_scores = []
ndcg_scores = []

for user, items in user_results.items():
    items.sort(reverse=True, key=lambda x: x[1])  # Sort based on prob_yes
    true_labels = [1 if item[2] == 'yes' else 0 for item in items]
    scores = [item[1] for item in items]

    if len(set(true_labels)) > 1:  # AUC is only valid if there are both positive and negative samples
        auc_scores.append(roc_auc_score(true_labels, scores))
    
    # Calculate nDCG@5
    relevance_scores = np.asarray([2 ** label - 1 for label in true_labels])
    ndcg_scores.append(ndcg_score([relevance_scores], [scores], k=5))

# Compute mean scores
mean_auc = np.mean(auc_scores)
mean_ndcg = np.mean(ndcg_scores)

print("Mean AUC:", mean_auc)
print("Mean nDCG@5:", mean_ndcg)


Mean AUC: 0.5855765996206029
Mean nDCG@5: 0.5915256829077676


In [17]:
import numpy as np

# Assuming results[0] has the following structure: user_id, impress_id, item_id, target_text, prob_yes, generated_sents
user_ids, impress_ids, item_ids, target_texts, prob_yes, generated_sents = results[0]

# Group data by user for MRR calculation
user_results = {}
for user, item, prob, response in zip(user_ids, item_ids, prob_yes, target_texts):
    if user not in user_results:
        user_results[user] = []
    user_results[user].append((item, prob, response))

# Calculate MRR
ranks = []
for user, result in user_results.items():
    # Sort results by descending probability score
    result.sort(reverse=True, key=lambda x: x[1])
    # Find the rank of the first correct answer
    for idx, result in enumerate(result):
        if result[2] == 'yes':  # Assuming 'yes' is the correct answer
            ranks.append(1 / (idx + 1))
            break

# Compute mean of reciprocal ranks
mrr = np.mean(ranks)
print("MRR:", mrr)

MRR: 0.8333333333333334
